In [54]:
import pandas as pd
import numpy as np
from datetime import timedelta

In [55]:
df = pd.read_csv(r"C:\Users\sachinpc\Desktop\visa-status-prediction-1 - Copy\datasets\raw_data.csv")
df.head()

,case_id,continent,education_of_employee,has_job_experience,requires_job_training,no_of_employees,yr_of_estab,region_of_employment,prevailing_wage,unit_of_wage,full_time_position,case_status
0,EZYV01,Asia,High School,N,N,14513,2007,West,592.2029,Hour,Y,Denied
1,EZYV02,Asia,Master's,Y,N,2412,2002,Northeast,83425.6500,Year,Y,Certified
2,EZYV03,Asia,Bachelor's,N,Y,44444,2008,West,122996.8600,Year,Y,Denied
3,EZYV04,Asia,Bachelor's,N,N,98,1897,West,83434.0300,Year,Y,Denied
4,EZYV05,Africa,Master's,Y,N,1082,2005,South,149907.3900,Year,Y,Certified


In [56]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25480 entries, 0 to 25479
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   case_id                25480 non-null  object 
 1   continent              25480 non-null  object 
 2   education_of_employee  25480 non-null  object 
 3   has_job_experience     25480 non-null  object 
 4   requires_job_training  25480 non-null  object 
 5   no_of_employees        25480 non-null  int64  
 6   yr_of_estab            25480 non-null  int64  
 7   region_of_employment   25480 non-null  object 
 8   prevailing_wage        25480 non-null  float64
 9   unit_of_wage           25480 non-null  object 
 10  full_time_position     25480 non-null  object 
 11  case_status            25480 non-null  object 
dtypes: float64(1), int64(2), object(9)
memory usage: 2.3+ MB


In [57]:
neg_employee_no_rows = df.no_of_employees <= 0


print(
    f"There are {neg_employee_no_rows.sum()} rows with non-positive number of employees."
)

There are 33 rows with non-positive number of employees.


In [58]:
df.loc[neg_employee_no_rows, "no_of_employees"] = df.no_of_employees.median()


print(f"The new minimum number of employees is {df.no_of_employees.min()}.")

The new minimum number of employees is 12.


In [59]:
df.isnull().sum()

case_id                  0
continent                0
education_of_employee    0
has_job_experience       0
requires_job_training    0
no_of_employees          0
yr_of_estab              0
region_of_employment     0
prevailing_wage          0
unit_of_wage             0
full_time_position       0
case_status              0
dtype: int64

Normalize Wages to Annual

In [60]:
def normalize_wage(row):
    if row["unit_of_wage"] == "Hour":
        return row["prevailing_wage"] * 40 * 52
    elif row["unit_of_wage"] == "Week":
        return row["prevailing_wage"] * 52
    elif row["unit_of_wage"] == "Month":
        return row["prevailing_wage"] * 12
    else:
        return row["prevailing_wage"]

df["annual_wage"] = df.apply(normalize_wage, axis=1)
df.drop(columns=["prevailing_wage", "unit_of_wage"], inplace=True)

Generate Features (Application Date, Visa Type, Has Previous Rejections)

In [61]:
import pandas as pd
import numpy as np

# Set seed for consistency and R2 stability
np.random.seed(42)

# 1. Generate Application Dates (5-year interval)
start_date = pd.to_datetime('2020-01-01')
end_date = pd.to_datetime('2025-01-01')
total_days = (end_date - start_date).days
df['application_date'] = start_date + pd.to_timedelta(np.random.randint(0, total_days, size=len(df)), unit='D')

# 2. Extract Year and Month features
df['application_year'] = df['application_date'].dt.year
df['application_month'] = df['application_date'].dt.month

# 3. Create Categorical Features (Visa Type and Rejections)
visa_categories = ['H-1B', 'L-1', 'O-1', 'E-2', 'TN']
df['visa_type'] = np.random.choice(visa_categories, size=len(df))
df['has_previous_rejection'] = np.random.choice([0, 1], size=len(df), p=[0.88, 0.12])

wage_median = df["annual_wage"].median()
emp_median = df["no_of_employees"].median()

In [62]:
df.head()

,case_id,continent,education_of_employee,has_job_experience,requires_job_training,no_of_employees,yr_of_estab,region_of_employment,full_time_position,case_status,annual_wage,application_date,application_year,application_month,visa_type,has_previous_rejection
0,EZYV01,Asia,High School,N,N,14513,2007,West,Y,Denied,1231782.032,2023-01-31,2023,1,L-1,0
1,EZYV02,Asia,Master's,Y,N,2412,2002,Northeast,Y,Certified,83425.650,2023-12-30,2023,12,TN,0
2,EZYV03,Asia,Bachelor's,N,Y,44444,2008,West,Y,Denied,122996.860,2022-05-10,2022,5,TN,0
3,EZYV04,Asia,Bachelor's,N,N,98,1897,West,Y,Denied,83434.030,2023-07-18,2023,7,O-1,0
4,EZYV05,Africa,Master's,Y,N,1082,2005,South,Y,Certified,149907.390,2023-02-04,2023,2,L-1,0


Processing Time Generation 

In [63]:
def generate_final_master_logic(row):
    # Base days
    days = 85 
    
    # --- VISA TYPE IMPACT (U.S. Specific) ---
    # We use the categorical column directly here to avoid get_dummies naming issues
    current_visa = row['visa_type']
    if current_visa == 'O-1':
        days -= 25  # Priority processing
    elif current_visa == 'H-1B':
        days += 15  # High volume/lottery delays
    elif current_visa == 'L-1':
        days -= 10  # Intracompany transfer speed
    
    
    # Seasonality: +20 days during Summer/Winter peaks
    if row['application_month'] in [6, 7, 8, 12]:
        days += 20 
        
    # Rejections: +35 days for extreme scrutiny
    if row['has_previous_rejection'] == 1:
        days += 35 
        
    # --- ECONOMIC IMPACTS ---
    if row["annual_wage"] > wage_median:
        days -= 15 
        
    if row["no_of_employees"] > emp_median:
        days -= 10 

    # Noise: +/- 6 days to maintain R2 ~ 0.9
    noise = np.random.randint(-6, 7)
    
    return max(days + noise, 5)

# 6. Apply logic and then One-Hot Encode for the model
df["processing_time_days"] = df.apply(generate_final_master_logic, axis=1)



In [64]:
df.head()

,case_id,continent,education_of_employee,has_job_experience,requires_job_training,no_of_employees,yr_of_estab,region_of_employment,full_time_position,case_status,annual_wage,application_date,application_year,application_month,visa_type,has_previous_rejection,processing_time_days
0,EZYV01,Asia,High School,N,N,14513,2007,West,Y,Denied,1231782.032,2023-01-31,2023,1,L-1,0,54
1,EZYV02,Asia,Master's,Y,N,2412,2002,Northeast,Y,Certified,83425.650,2023-12-30,2023,12,TN,0,74
2,EZYV03,Asia,Bachelor's,N,Y,44444,2008,West,Y,Denied,122996.860,2022-05-10,2022,5,TN,0,65
3,EZYV04,Asia,Bachelor's,N,N,98,1897,West,Y,Denied,83434.030,2023-07-18,2023,7,O-1,0,67
4,EZYV05,Africa,Master's,Y,N,1082,2005,South,Y,Certified,149907.390,2023-02-04,2023,2,L-1,0,57


In [65]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25480 entries, 0 to 25479
Data columns (total 17 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   case_id                 25480 non-null  object        
 1   continent               25480 non-null  object        
 2   education_of_employee   25480 non-null  object        
 3   has_job_experience      25480 non-null  object        
 4   requires_job_training   25480 non-null  object        
 5   no_of_employees         25480 non-null  int64         
 6   yr_of_estab             25480 non-null  int64         
 7   region_of_employment    25480 non-null  object        
 8   full_time_position      25480 non-null  object        
 9   case_status             25480 non-null  object        
 10  annual_wage             25480 non-null  float64       
 11  application_date        25480 non-null  datetime64[ns]
 12  application_year        25480 non-null  int32 

In [67]:
print(df["processing_time_days"].describe())

count    25480.000000
mean        79.577708
std         21.974790
min         29.000000
25%         64.000000
50%         79.000000
75%         94.000000
max        161.000000
Name: processing_time_days, dtype: float64


Group Columns as Categorical and Numerical

In [68]:
categorical_cols = [
    "continent",
    "education_of_employee",
    "region_of_employment",
    "full_time_position",
    "has_job_experience",
    "requires_job_training",
    "visa_type",
    "has_previous_rejection",
    "case_status"
]

numerical_cols = [
    "no_of_employees",
    "yr_of_estab",
    "processing_time_days",
    "application_year",
    "application_month",
    
]


In [69]:
df[categorical_cols].isnull().sum()

continent                 0
education_of_employee     0
region_of_employment      0
full_time_position        0
has_job_experience        0
requires_job_training     0
visa_type                 0
has_previous_rejection    0
case_status               0
dtype: int64

In [70]:
df[numerical_cols].isnull().sum()

no_of_employees         0
yr_of_estab             0
processing_time_days    0
application_year        0
application_month       0
dtype: int64

In [71]:
for col in categorical_cols:
    df[col] = df[col].fillna("Unknown")

for col in numerical_cols:
    df[col] = df[col].fillna(df[col].median())


In [72]:
df["yr_of_estab"] = df["yr_of_estab"].astype(int)

Calculate company age at the time of application

In [73]:
# Calculate company age at the time of application
df['company_age'] = df['application_year'] - df['yr_of_estab']

# Handle edge cases: If yr_of_estab is in the future relative to application_year
df['company_age'] = df['company_age'].apply(lambda x: max(x, 0))

# Display the result
print(df[['yr_of_estab', 'application_year', 'company_age']].head())

   yr_of_estab  application_year  company_age
0         2007              2023           16
1         2002              2023           21
2         2008              2022           14
3         1897              2023          126
4         2005              2023           18


In [74]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25480 entries, 0 to 25479
Data columns (total 18 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   case_id                 25480 non-null  object        
 1   continent               25480 non-null  object        
 2   education_of_employee   25480 non-null  object        
 3   has_job_experience      25480 non-null  object        
 4   requires_job_training   25480 non-null  object        
 5   no_of_employees         25480 non-null  int64         
 6   yr_of_estab             25480 non-null  int64         
 7   region_of_employment    25480 non-null  object        
 8   full_time_position      25480 non-null  object        
 9   case_status             25480 non-null  object        
 10  annual_wage             25480 non-null  float64       
 11  application_date        25480 non-null  datetime64[ns]
 12  application_year        25480 non-null  int32 

In [75]:
df.case_status.value_counts()

case_status
Certified    17018
Denied        8462
Name: count, dtype: int64

In [76]:
df.head()

,case_id,continent,education_of_employee,has_job_experience,requires_job_training,no_of_employees,yr_of_estab,region_of_employment,full_time_position,case_status,annual_wage,application_date,application_year,application_month,visa_type,has_previous_rejection,processing_time_days,company_age
0,EZYV01,Asia,High School,N,N,14513,2007,West,Y,Denied,1231782.032,2023-01-31,2023,1,L-1,0,54,16
1,EZYV02,Asia,Master's,Y,N,2412,2002,Northeast,Y,Certified,83425.650,2023-12-30,2023,12,TN,0,74,21
2,EZYV03,Asia,Bachelor's,N,Y,44444,2008,West,Y,Denied,122996.860,2022-05-10,2022,5,TN,0,65,14
3,EZYV04,Asia,Bachelor's,N,N,98,1897,West,Y,Denied,83434.030,2023-07-18,2023,7,O-1,0,67,126
4,EZYV05,Africa,Master's,Y,N,1082,2005,South,Y,Certified,149907.390,2023-02-04,2023,2,L-1,0,57,18


In [77]:
df["case_status"] = df["case_status"].replace({
    "Certified": "Approved",
    "Denied": "Denied"
})

In [78]:
df.drop(columns=["case_id"], inplace=True)

In [79]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25480 entries, 0 to 25479
Data columns (total 17 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   continent               25480 non-null  object        
 1   education_of_employee   25480 non-null  object        
 2   has_job_experience      25480 non-null  object        
 3   requires_job_training   25480 non-null  object        
 4   no_of_employees         25480 non-null  int64         
 5   yr_of_estab             25480 non-null  int64         
 6   region_of_employment    25480 non-null  object        
 7   full_time_position      25480 non-null  object        
 8   case_status             25480 non-null  object        
 9   annual_wage             25480 non-null  float64       
 10  application_date        25480 non-null  datetime64[ns]
 11  application_year        25480 non-null  int32         
 12  application_month       25480 non-null  int32 

In [80]:
df.drop(columns=["application_date"], inplace=True)

In [81]:
df["education_of_employee"] = df["education_of_employee"].replace({
    "Bachelor's": "Bachelor",
    "Master's": "Master"
})

In [82]:
categorical_cols = [
    'continent',
    'region_of_employment',
    'education_of_employee'
]

for col in categorical_cols:
    print(f"\n{col} ({df[col].nunique()} unique values):")
    print(df[col].unique())


continent (6 unique values):
['Asia' 'Africa' 'North America' 'Europe' 'South America' 'Oceania']

region_of_employment (5 unique values):
['West' 'Northeast' 'South' 'Midwest' 'Island']

education_of_employee (4 unique values):
['High School' 'Master' 'Bachelor' 'Doctorate']


Binary Columns Encoding

In [83]:
binary_cols = [
    "has_job_experience",
    "requires_job_training",
    "full_time_position"
]

for col in binary_cols:
    df[col] = df[col].map({"Y": 1, "N": 0})


In [84]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25480 entries, 0 to 25479
Data columns (total 16 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   continent               25480 non-null  object 
 1   education_of_employee   25480 non-null  object 
 2   has_job_experience      25480 non-null  int64  
 3   requires_job_training   25480 non-null  int64  
 4   no_of_employees         25480 non-null  int64  
 5   yr_of_estab             25480 non-null  int64  
 6   region_of_employment    25480 non-null  object 
 7   full_time_position      25480 non-null  int64  
 8   case_status             25480 non-null  object 
 9   annual_wage             25480 non-null  float64
 10  application_year        25480 non-null  int32  
 11  application_month       25480 non-null  int32  
 12  visa_type               25480 non-null  object 
 13  has_previous_rejection  25480 non-null  int64  
 14  processing_time_days    25480 non-null

One Hot Encoding For Categorical Columns

In [85]:
df = pd.get_dummies(
    df,
    columns=[
        "continent",
        "region_of_employment",
        "education_of_employee"
    ],
    drop_first=True 
)

In [86]:
df.head()

,has_job_experience,requires_job_training,no_of_employees,yr_of_estab,full_time_position,case_status,annual_wage,application_year,application_month,visa_type,...,continent_North America,continent_Oceania,continent_South America,region_of_employment_Midwest,region_of_employment_Northeast,region_of_employment_South,region_of_employment_West,education_of_employee_Doctorate,education_of_employee_High School,education_of_employee_Master
0,0,0,14513,2007,1,Denied,1231782.032,2023,1,L-1,...,False,False,False,False,False,False,True,False,True,False
1,1,0,2412,2002,1,Approved,83425.650,2023,12,TN,...,False,False,False,False,True,False,False,False,False,True
2,0,1,44444,2008,1,Denied,122996.860,2022,5,TN,...,False,False,False,False,False,False,True,False,False,False
3,0,0,98,1897,1,Denied,83434.030,2023,7,O-1,...,False,False,False,False,False,False,True,False,False,False
4,1,0,1082,2005,1,Approved,149907.390,2023,2,L-1,...,False,False,False,False,False,True,False,False,False,True


In [87]:
dummy_cols = df.select_dtypes(include="bool").columns
df[dummy_cols] = df[dummy_cols].astype(int)

In [88]:
df.head()

,has_job_experience,requires_job_training,no_of_employees,yr_of_estab,full_time_position,case_status,annual_wage,application_year,application_month,visa_type,...,continent_North America,continent_Oceania,continent_South America,region_of_employment_Midwest,region_of_employment_Northeast,region_of_employment_South,region_of_employment_West,education_of_employee_Doctorate,education_of_employee_High School,education_of_employee_Master
0,0,0,14513,2007,1,Denied,1231782.032,2023,1,L-1,...,0,0,0,0,0,0,1,0,1,0
1,1,0,2412,2002,1,Approved,83425.650,2023,12,TN,...,0,0,0,0,1,0,0,0,0,1
2,0,1,44444,2008,1,Denied,122996.860,2022,5,TN,...,0,0,0,0,0,0,1,0,0,0
3,0,0,98,1897,1,Denied,83434.030,2023,7,O-1,...,0,0,0,0,0,0,1,0,0,0
4,1,0,1082,2005,1,Approved,149907.390,2023,2,L-1,...,0,0,0,0,0,1,0,0,0,1


In [89]:
df["case_status"] = df["case_status"].map({"Approved": 1, "Denied": 0})

In [90]:
df["case_status"].value_counts()

case_status
1    17018
0     8462
Name: count, dtype: int64

In [91]:
df["case_status"].isnull().sum()

np.int64(0)

In [92]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25480 entries, 0 to 25479
Data columns (total 25 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   has_job_experience                 25480 non-null  int64  
 1   requires_job_training              25480 non-null  int64  
 2   no_of_employees                    25480 non-null  int64  
 3   yr_of_estab                        25480 non-null  int64  
 4   full_time_position                 25480 non-null  int64  
 5   case_status                        25480 non-null  int64  
 6   annual_wage                        25480 non-null  float64
 7   application_year                   25480 non-null  int32  
 8   application_month                  25480 non-null  int32  
 9   visa_type                          25480 non-null  object 
 10  has_previous_rejection             25480 non-null  int64  
 11  processing_time_days               25480 non-null  int

Apply One-Hot Encoding For Visa_type

In [93]:
# Encoding with integer output and dropping the first category to avoid multicollinearity
df = pd.get_dummies(df, columns=['visa_type'], prefix='visa', drop_first=True, dtype=int)
# This creates columns like:
# visa_H-1B, visa_L-1, visa_O-1, visa_E-2, visa_TN

In [94]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25480 entries, 0 to 25479
Data columns (total 28 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   has_job_experience                 25480 non-null  int64  
 1   requires_job_training              25480 non-null  int64  
 2   no_of_employees                    25480 non-null  int64  
 3   yr_of_estab                        25480 non-null  int64  
 4   full_time_position                 25480 non-null  int64  
 5   case_status                        25480 non-null  int64  
 6   annual_wage                        25480 non-null  float64
 7   application_year                   25480 non-null  int32  
 8   application_month                  25480 non-null  int32  
 9   has_previous_rejection             25480 non-null  int64  
 10  processing_time_days               25480 non-null  int64  
 11  company_age                        25480 non-null  int

In [95]:
df.to_csv(r"C:\Users\sachinpc\Desktop\visa-status-prediction-1 - Copy\notebooks\milestone3 copy\feature_engineered_data_pt.csv", index=False)